<a href="https://colab.research.google.com/github/hilmimughid/PCVK_Ganjil_2023/blob/main/Week_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nama: Hilmi Mughid**

# **Kelas: TI-3C**

# **No. Absen: 09**

# **NIM: 2141720081**